In this notebook we will create word2vec fron our training dataset.

In [1]:
import pandas as pd
import nltk
import os
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
import re


In [2]:
print(os.listdir('./data'))

['labeledTrainData.tsv', 'testData.tsv', 'unlabeledTrainData.tsv']


In [6]:
#first we are going to read data 
labeled_train = pd.read_csv('./data/labeledTrainData.tsv',delimiter='\t',quoting=3)
unlabeled_train = pd.read_csv('./data/unlabeledTrainData.tsv',delimiter='\t',quoting=3)
print(labeled_train.columns.values)
print(unlabeled_train.columns.values)

['id' 'sentiment' 'review']
['id' 'review']


In [21]:
#defining a function to convert reviews to list of words
#stop_words=stopwords.words("english")
def review_to_listofwords(raw_review):
    #stripping html out of review
    review_text=BeautifulSoup(raw_review,"html.parser").getText()
    #getting rid of all the punctuation
    review_text=re.sub("[^a-zA-Z]", " ", review_text) 
    #converting in to lower case
    review_text=review_text.lower()
    #tokenizing to words
    words=review_text.split()
    return words  

Word2Vec expects single sentences, each one as a list of words.

In [22]:
#function converts a review into list of sentences where each sentence is composed of a list of words
def review_to_sentences(review,tokenizer):
    raw_sentences = tokenizer.tokenize(review.strip())
    sentences = []
    for s in raw_sentences:
        if len(s) > 0:
            sentences.append(review_to_listofwords(s))
    return sentences        

In [23]:
def build_dataset(dataframe,tokenizer):
    train_reviews = dataframe['review']
    sentences=[]
    for review in train_reviews:
        sentences=sentences+review_to_sentences(review,tokenizer)
    return sentences    

In [24]:
#we will tokenize our reviews into sentences using punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
labeled_sentences=build_dataset(labeled_train,tokenizer)
unlabeled_sentences=build_dataset(labeled_train,tokenizer)
sentences=labeled_sentences+unlabeled_sentences

/home/hardik/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/home/hardik/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


<B>Hyperparameters for word2vec model</B>:<br> <br>
Architecture: Architecture options are skip-gram (default) or continuous bag of words. We found that skip-gram was very slightly slower but produced better results. <br> <br>
Training algorithm: Hierarchical softmax (default) or negative sampling. For us, the default worked well.
Downsampling of frequent words: The Google documentation recommends values between .00001 and .001. For us, values closer 0.001 seemed to improve the accuracy of the final model. <br> <br>
Word vector dimensionality: More features result in longer runtimes, and often, but not always, result in better models. Reasonable values can be in the tens to hundreds; we used 300.<br> <br>
Context / window size: How many words of context should the training algorithm take into account? 10 seems to work well for hierarchical softmax (more is better, up to a point).<br> <br>
Worker threads: Number of parallel processes to run. This is computer-specific, but between 4 and 6 should work on most systems.<br> <br>
Minimum word count: This helps limit the size of the vocabulary to meaningful words. Any word that does not occur at least this many times across all documents is ignored. Reasonable values could be between 10 and 100. In this case, since each movie occurs 30 times, we set the minimum word count to 40, to avoid attaching too much importance to individual movie titles. This resulted in an overall vocabulary size of around 15,000 words. Higher values also help limit run time.

In [25]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

In [26]:
# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

In [28]:
from gensim.models import word2vec
model = word2vec.Word2Vec(sentences, workers=num_workers,
                          size=num_features, min_count = min_word_count,
                          window = context, sample = downsampling)

2018-01-09 22:46:09,213 : INFO : 'pattern' package not found; tag filters are not available for English
2018-01-09 22:46:09,219 : INFO : collecting all words and their counts
2018-01-09 22:46:09,220 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-01-09 22:46:09,287 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types
2018-01-09 22:46:09,342 : INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types
2018-01-09 22:46:09,399 : INFO : PROGRESS: at sentence #30000, processed 671314 words, keeping 30034 word types
2018-01-09 22:46:09,454 : INFO : PROGRESS: at sentence #40000, processed 897814 words, keeping 34348 word types
2018-01-09 22:46:09,511 : INFO : PROGRESS: at sentence #50000, processed 1116962 words, keeping 37761 word types
2018-01-09 22:46:09,567 : INFO : PROGRESS: at sentence #60000, processed 1338403 words, keeping 40723 word types
2018-01-09 22:46:09,624 : INFO : PROGRESS: at senten

2018-01-09 22:46:23,015 : INFO : PROGRESS: at 13.67% examples, 568287 words/s, in_qsize 7, out_qsize 0
2018-01-09 22:46:24,018 : INFO : PROGRESS: at 15.04% examples, 568554 words/s, in_qsize 7, out_qsize 0
2018-01-09 22:46:25,022 : INFO : PROGRESS: at 16.42% examples, 569292 words/s, in_qsize 7, out_qsize 0
2018-01-09 22:46:26,023 : INFO : PROGRESS: at 17.81% examples, 570577 words/s, in_qsize 7, out_qsize 0
2018-01-09 22:46:27,036 : INFO : PROGRESS: at 19.19% examples, 570704 words/s, in_qsize 7, out_qsize 0
2018-01-09 22:46:28,045 : INFO : PROGRESS: at 20.59% examples, 571438 words/s, in_qsize 7, out_qsize 0
2018-01-09 22:46:29,052 : INFO : PROGRESS: at 21.97% examples, 571747 words/s, in_qsize 7, out_qsize 0
2018-01-09 22:46:30,062 : INFO : PROGRESS: at 23.35% examples, 571911 words/s, in_qsize 7, out_qsize 0
2018-01-09 22:46:31,067 : INFO : PROGRESS: at 24.74% examples, 572232 words/s, in_qsize 7, out_qsize 0
2018-01-09 22:46:32,079 : INFO : PROGRESS: at 26.15% examples, 572623 wor

In [30]:
# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)
model_name = "300features_40minwords_10context"
model.save(model_name)

2018-01-09 22:54:46,286 : INFO : precomputing L2-norms of word weight vectors
2018-01-09 22:54:46,437 : INFO : saving Word2Vec object under 300features_40minwords_10context, separately None
2018-01-09 22:54:46,438 : INFO : not storing attribute syn0norm
2018-01-09 22:54:46,439 : INFO : not storing attribute cum_table
2018-01-09 22:54:46,995 : INFO : saved 300features_40minwords_10context


In [41]:
model.most_similar("excellent")

/home/hardik/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('outstanding', 0.7961982488632202),
 ('exceptional', 0.7388363480567932),
 ('superb', 0.7252383828163147),
 ('terrific', 0.6861687898635864),
 ('fantastic', 0.6683037281036377),
 ('amazing', 0.6594833731651306),
 ('brilliant', 0.6451787948608398),
 ('awesome', 0.6322755813598633),
 ('incredible', 0.6180087327957153),
 ('marvelous', 0.617539644241333)]

In [43]:
model.most_similar("atrocious")

/home/hardik/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('abysmal', 0.7746913433074951),
 ('horrendous', 0.7522616982460022),
 ('appalling', 0.7396042346954346),
 ('terrible', 0.7288037538528442),
 ('amateurish', 0.7272131443023682),
 ('horrid', 0.71351158618927),
 ('laughable', 0.7053291201591492),
 ('awful', 0.7022461891174316),
 ('uninspired', 0.6944233775138855),
 ('dreadful', 0.6936764121055603)]

In [46]:
model.most_similar("lovely")

/home/hardik/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('gorgeous', 0.6907073259353638),
 ('beautiful', 0.6399940252304077),
 ('elegant', 0.6288249492645264),
 ('delightful', 0.6285260915756226),
 ('radiant', 0.6140733957290649),
 ('barbara', 0.613771378993988),
 ('ellen', 0.6056407690048218),
 ('sweet', 0.6018452048301697),
 ('clara', 0.5908852219581604),
 ('charming', 0.58653724193573)]